**Визуализация и Кластеризация данных: Отчет по мировым показателям счастья.**

Показана базовая визуализация с использованием карты хороплет. Далее произведена кластеризация с использованием алгоритма k-means на нескольких факторах, таких как ВВП на душу населения, ожидаемая продолжительность жизни, коррупция и так далее. Данные взяты за 2017 год.  

In [242]:
#Call required libraries
import time                   # To time processes
import warnings               # To suppress warnings

import numpy as np            # Data manipulation
import pandas as pd           # Dataframe manipulatio 
import matplotlib.pyplot as plt                   # For graphics
import seaborn as sns
import plotly.plotly as py #For World Map
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from sklearn.preprocessing import StandardScaler  # For scaling dataset
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation #For clustering
from sklearn.mixture import GaussianMixture #For GMM clustering

import os                     # For os related operations
import sys                    # For data size


In [243]:
wh = pd.read_csv("../input/2017.csv") #Read the dataset
wh.describe()

In [244]:
wh.head()

In [245]:
print("Dimension of dataset: wh.shape")
wh.dtypes

**Визуализация данных**

*Зависимости между переменными*

Для начала, найдем зависимости между несколькими переменными. Вычислим корреляционную матрицу между переменными и отобразим их на тепловой карте.

In [246]:
wh1 = wh[['Happiness.Score','Economy..GDP.per.Capita.','Family','Health..Life.Expectancy.', 'Freedom', 
          'Generosity','Trust..Government.Corruption.','Dystopia.Residual']] #Subsetting the data
cor = wh1.corr() #Calculate the correlation of the above variables
sns.heatmap(cor, square = True) #Plot the correlation as heat map

Мы получили тепловую карту корреляции между переменными. Чем светлее цвет на тепловой карте, тем выше зависимость. Можем заметить, что показатели счастья имеют сильную корреляцию с ВВП на душу населения, семьей и ожидаемой продолжительностью жизни и имеет наименьшую корреляцию с показателями щедрости. 

*Визуализация показателей счастья с использованием карты хороплет*

Выполним визуализацию показателей счастья различных стран на карте мира. Наведение мыши на страну покажет нам имя страны и значение показателя счастья для данной страны. 

In [247]:
#Ref: https://plot.ly/python/choropleth-maps/
data = dict(type = 'choropleth', 
           locations = wh['Country'],
           locationmode = 'country names',
           z = wh['Happiness.Score'], 
           text = wh['Country'],
           colorbar = {'title':'Happiness'})
layout = dict(title = 'Happiness Index 2017', 
             geo = dict(showframe = False, 
                       projection = {'type': 'Mercator'}))
choromap3 = go.Figure(data = [data], layout=layout)
iplot(choromap3)

**Кластеризация стран**

Мы рассматриваем восемь параметров, а именно: оценку счастья, ВВП на душу населения, семью, ожидаемую продолжительность жизни, свободу, щедрость, коррупцию и дистопию для кластеризации стран. Поскольку кластеризация чувствительна к диапазону данных, желательно масштабировать данные перед продолжением.

In [248]:
#Scaling of data
wh2 = wh[['Economy..GDP.per.Capita.','Trust..Government.Corruption.']]
ss = StandardScaler()
ss.fit_transform(wh2)

**Кластеризация k-средних**

В данном алгоритме необходимо пределить количество кластеров. Кластеры формируются на основе близости к центральному значению кластеров. Начальное значение центра выбирается случайным образом. Кластеризация K-средних - это подход сверху вниз, в том смысле, мы определяем количество кластеров (k), а затем группируем точки данных в k кластеров.

In [249]:
#K means Clustering 
def doKmeans(X, nclust=2):
    model = KMeans(nclust)
    model.fit(X)
    clust_labels = model.predict(X)
    cent = model.cluster_centers_
    return (clust_labels, cent)

clust_labels, centroids = doKmeans(wh2, 3)
kmeans = pd.DataFrame(clust_labels)
wh2.insert((wh2.shape[1]),'kmeans',kmeans)
print(centroids)


In [250]:
#Plot the clusters obtained using k means
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(wh1['Economy..GDP.per.Capita.'],wh1['Trust..Government.Corruption.'],
                     c=kmeans[0],s=50)
ax.scatter(centroids[:,0],centroids[:,1], color='red', s=120)  
ax.set_title('K-Means Clustering')
ax.set_xlabel('GDP per Capita')
ax.set_ylabel('Corruption')
plt.colorbar(scatter)